In [1]:
# surpress future warning related to SkLearn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import tensorflow as tf
import keras
import numpy as np

from keras.src.layers import Embedding
from keras.layers import Dense
from keras.src.utils import to_categorical
from keras.models import Sequential

from scipy.sparse import csr_matrix

from numpy.lib.stride_tricks import sliding_window_view
from numpy import argmax, array


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.utils import compute_sample_weight

from imblearn.over_sampling import RandomOverSampler
from tqdm.keras import TqdmCallback
from imblearn.over_sampling import SMOTENC

from keras.layers import Bidirectional,Dense, Embedding, LSTM, SpatialDropout1D,BatchNormalization,GaussianNoise
from keras import Input
from keras.metrics import F1Score



/Users/seb/git/master_thesis/nw_event_abstraction/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df_r4 = pd.read_csv('../data_mod/R4_Train_poslabel.csv')
df_r4_interleaved = pd.read_csv('../data_mod/R4_HR_Interleaved_poslabel.csv')

df_r1 = pd.read_csv('../data_mod/R1_Train_poslabel.csv')
df_r1_interleaved = pd.read_csv('../data_mod/R1_HR_Interleaved_poslabel.csv')

df2_r4 = pd.read_csv('../data_mod/R4/R4_Valid.csv')

In [6]:
from itertools import product

def sliding_window(observation, forward, backward): 
    window_size = forward+backward+1
    sequences = sliding_window_view(observation, window_size)
    
    # pad first n-1 elements
    #pad_top = [[0] * (sequence_length - 1 - i) + sequences[0, 0:i + 1].tolist() for i in range(sequence_length - 1)]
    
    #res = np.insert(sequences,0, pad_top, axis=0)
    #res = np.asarray(X).astype(np.float32) 
    #res = res.reshape(res.shape[0], res.shape[1], 1)
    return sequences


def generate_sequences(observation,end_indices, start_indices, forward, backward, label_encoder):
    # window size for stubs is 1 smaller than regular window size
    window_size = forward+backward
    # generate a sliding window over the observation with the new window size
    seq = sliding_window_view(observation, window_size)
    
    # get all windows that end on "position_end" as indicated by end_indices
    end_windows = [seq[i-window_size+1 ] for i in end_indices]
    
    # get all windows that START on "position_start" as indicated by start_indices
    start_windows = [seq[i] for i in start_indices]
    
    # generate all combinations of ...end, start... 
    flattened_combinations = [np.concatenate(combination).tolist() for combination in product(end_windows, start_windows)]
    
    # generate new sequences by applying the sliding window over the new combinations
    sequences = np.concatenate([sliding_window_view(c, window_size+1).tolist() for c in flattened_combinations])

    
    labels = label_encoder.transform(["position_between"] * (window_size-1) + ["position_end"] + ["position_start"] + ["position_between"] * (window_size-1))
    labels = to_categorical(np.tile(labels[backward: -forward], len(flattened_combinations)))
    
    return sequences, labels

def remove_duplicates(features, labels): 
    data = np.column_stack((features, labels))
    unique_data = np.unique(data, axis = 0)
    unique_train_features = unique_data[:,:-3]
    unique_train_labels = unique_data[:,-3:]
    
    print("unique labels", np.unique(unique_train_labels, axis = 0, return_counts=True))
    
    return unique_train_features, unique_train_labels

def preprocess_data(train_features,train_labels, valid_features, valid_labels,forward_window, backward_window):
    # encode message type for both training and validation data
    message_type_encoder = LabelEncoder().fit(train_features.values)
    train_features_encoded = message_type_encoder.transform(train_features) +1
    valid_features_encoded = message_type_encoder.transform(valid_features) +1 

    # generate feature sequences 
    
    train_feature_sequence = sliding_window(train_features_encoded,forward_window, backward_window)
    valid_feature_sequence = sliding_window(valid_features_encoded, forward_window, backward_window)
    
    
    # generate more start / end overlaps 
    end_indices = train_labels[train_labels == "position_end"].index.tolist()
    start_indices = train_labels[train_labels == "position_start"].index.tolist()

    # one hot encode labels
    label_encoder = LabelEncoder().fit(train_labels)
    train_labels_encoded = to_categorical(label_encoder.transform(train_labels[backward_window: -forward_window]))
    valid_labels_encoded = to_categorical(label_encoder.transform(valid_labels[backward_window: -forward_window]))
    
    unique_x, unique_y = remove_duplicates(train_feature_sequence, train_labels_encoded)

    
    synth_sequences, synth_labels = generate_sequences(train_features_encoded, end_indices, start_indices, forward_window, backward_window, label_encoder)
 
    print(f"generated {len(synth_sequences)} additional samples")    
    
    train_feature_sequence = np.concatenate((train_feature_sequence, synth_sequences), axis = 0)
    train_labels_encoded = np.concatenate((train_labels_encoded, synth_labels), axis = 0)
        
    # remove duplicates
    features, labels = remove_duplicates(train_feature_sequence, train_labels_encoded)

    
    return features, labels, valid_feature_sequence, valid_labels_encoded, label_encoder.classes_.tolist(), len(message_type_encoder.classes_)
    

In [5]:
x, Y, x_il, Y_il, labels, label_length = preprocess_data(train_features = df_r4["event_with_roles"], train_labels = df_r4["task_position"],
                                           valid_features = df_r4_interleaved["event_with_roles"], valid_labels = df_r4_interleaved["task_position"],
                                           forward_window = 10, backward_window = 5)

X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size = 0.3, shuffle = True, random_state = 42)

unique labels (array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([ 20,  19, 449]))
generated 13057335 additional samples
unique labels (array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]]), array([ 77,  70, 875]))
1022
1022


In [7]:
np.unique(Y, return_counts=True)

(array([0., 1.]), array([2044, 1022]))

,0,1
0,0.0,1.0
1,2044.0,1022.0
